In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import GridspecLayout
import base64

def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(width='auto'))


def create_download_link_pt_br( df, title = "Download Formato Brasileiro", filename = "data.csv"):  
    csv = df.to_csv(sep=';', decimal=',')
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<b><a style="color:orange;" download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a></b>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

def create_download_link_us( df, title = "Download Formato Americano", filename = "data.csv"):  
    csv = df.to_csv(sep=',', decimal='.')
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<b><a style="color:blue;" download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a></b>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

In [12]:
# Pede a matrícula e recupera as informações de X e Y que serão usadas para a resolução do problema
# matricula = input('Qual a sua matrícula?')

matricula_l = widgets.HTML(value = 'Matricula:')
matricula = widgets.Text(
    value='',
    placeholder='',
#     description='Matricula:',
    disabled=False,
    layout=widgets.Layout(height='auto', width='auto')
)


button = create_expanded_button('Começar', 'warning')
button_r = create_expanded_button('Resetar', 'info')
button_r.disabled = True
output = widgets.Output(layout={'border': '1px solid black'},width='auto')

def on_button_clicked_r(b):
    button.disabled = False
    output.clear_output()
#     grid[1,:15] = output
    button_r.disabled = True
    output.layout.visibility = 'hidden'

def on_button_clicked(b):
    button.disabled = True
    button_r.disabled = False
    output.layout.visibility = 'visible'
#     grid[1,:15] = output
    with output:
        matri = matricula.value.replace('.','').replace('-','')
        DIG_X = int(matri[-2:][0:1])
        DIG_Y = int(matri[-1:])
        
        # Cria as tabelas fornecidas pelo enunciado
        df = {
            'NUM_MAQ_QUE_HR': [1,2,3,4,5,6,7,8],
            'NUM_OCOR': [22,42,94,124,82,36,20,4]
        }
        df = pd.DataFrame(df, index = range(1,9))


        df_X = {
            'DIG_X': [0,1,2,3,4,5,6,7,8,9],
            '$_HMP_QUEBRA': [22.2,21.6,20.2,22.4,21.8,20.5,22.6,21.2,20.8,20.4]
        }
        df_X = pd.DataFrame(df_X)


        df_Y = {
            'DIG_Y': [0,1,2,3,4,5,6,7,8,9],
            '$_HMP_RESER': [12,14,16,18,11,13,15,17,19,10]
        }
        df_Y = pd.DataFrame(df_Y)

        
        
        # Cria colunas e recupera informações da tabelas que serão usadas na solução

        df['FREQ'] = df['NUM_OCOR']/df['NUM_OCOR'].sum()
        C_HMP_QUEBRA = df_X[df_X['DIG_X'] == DIG_X].iloc[0,1]
        C_HMP_RESER = df_Y[df_Y['DIG_Y'] == DIG_Y].iloc[0,1]
        
        # Cria a matrix já com a relação de subtração entre os índices fornecidos de 1 a 8 para relação de máquina reserva e máquina quebrada
        v = np.array(df['NUM_MAQ_QUE_HR'])

        arr = np.array(v[:, None] - v)
        df_matrix = pd.DataFrame(arr, index = [1,2,3,4,5,6,7,8] ,columns = [1,2,3,4,5,6,7,8])
        display(widgets.HTML(value = '<b style="color:red;">Verifique sua matrícula: ' + 
                             matri[0:2] + '.' + matri[-7:][0:3] + '.' + matri[-4:][0:3] + '-' + matri[-1:] +
                             '</b><br><br>' +
                             '<b>Matrix Subtrativa |  </b>' +
                             create_download_link_pt_br(df_matrix) + ' <b> | </b> ' +
                             create_download_link_us(df_matrix)))
        display(df_matrix)
        display(widgets.HTML(value = '<br>'))
        
        # Relaciona o custo de quebra, custo de maquina reserva e multiplica pela frequência da tabela fornecida pelo enunciado.
        ## Cria-se a coluna com a somatória de custo total também

        df_custos_inter = df_matrix.copy()
        for col in df_custos_inter.columns:
            df_custos_inter[col] = list(map(lambda x: abs(x * C_HMP_RESER) if x > 0 else abs(x * C_HMP_QUEBRA), df_custos_inter[col]))
        
        
        display(widgets.HTML(value = '<b>Tabela intermediária de custos , sem multiplicar a ocorrência |  </b> ' +
                             create_download_link_pt_br(df_custos_inter) + ' <b> | </b>' +
                             create_download_link_us(df_custos_inter)))
        display(df_custos_inter)
        display(widgets.HTML(value = '<br>'))
        
        df_custos = df_matrix.copy()
        for col in df_custos.columns:
            df_custos[col] = list(map(lambda x: abs(x * C_HMP_RESER) if x > 0 else abs(x * C_HMP_QUEBRA), df_custos[col]))
            df_custos[col] = df['FREQ'].iloc[int(col)-1] * df_custos[col]

        df_custos['CUSTO_TOTAL'] = df_custos.sum(axis = 1, skipna = True)

        display(widgets.HTML(value = '<b>Tabela de custos finais, multiplicando a ocorrência |  </b> ' + create_download_link_pt_br(df_custos) + '<b> | </b> ' + create_download_link_us(df_custos)))
        
        display(df_custos)
        
        
        # Gera a resposta do problema

        C_Min = df_custos['CUSTO_TOTAL'].min()

        N_Maquinas_Reservas = df_custos.index[df_custos['CUSTO_TOTAL'] == df_custos['CUSTO_TOTAL'].min()].tolist()[0]
        N_Maquinas_Reservas_Final = N_Maquinas_Reservas

        for a in range(N_Maquinas_Reservas + 1, df_custos['CUSTO_TOTAL'].count()+1):

            if df_custos['CUSTO_TOTAL'][a] - C_Min < C_HMP_RESER:
                N_Maquinas_Reservas_Final = a
            else:
                N_Maquinas_Reservas_Final = N_Maquinas_Reservas_Final

        if N_Maquinas_Reservas == N_Maquinas_Reservas_Final:
            text = ('Serão alocadas ' +
                  str(N_Maquinas_Reservas_Final) +
                  ' máquinas reservas com o custo total de $' +
                  "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final],2)).replace('.',',') +
                  ' com a cobertura das ocorrências de ' +
                  "{:.2f}".format(round(df['FREQ'][df['NUM_MAQ_QUE_HR'] <= N_Maquinas_Reservas_Final].sum()*100,2)).replace('.',',') +
                  '%. Não houve a necessidade de alocar mais máquinas, pois o custo de alocação de mais uma máquina foi maior do que o custo da máquina reserva[$' +
                  "{:.2f}".format(round(float(C_HMP_RESER),2)).replace('.',',') +
                  '].')
        else:
            text = ('Serão alocadas ' +
                  str(N_Maquinas_Reservas_Final) +
                  ' máquinas reservas com o custo total de $' +
                  "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final],2)).replace('.',',') +
                  ' com a cobertura das ocorrências de ' +
                  str(round(df['FREQ'][df['NUM_MAQ_QUE_HR'] <= N_Maquinas_Reservas_Final].sum()*100,2)).replace('.',',') +
                  '%. Foram alocadas ' +
                  str(N_Maquinas_Reservas_Final-N_Maquinas_Reservas) +
                  ' máquinas a mais em relação ao custo mínimo da tabela, isto foi feito devido a diferença de custo de alocação dessas máquinas com custo mínimo [$' +
                  "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final] - df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas],2)).replace('.',',') +
                  '] ser menor do que o custo de uma máquina reserva [$' +
                  "{:.2f}".format(round(float(C_HMP_RESER),2)).replace('.',',') +
                  ']. O custo mínimo ocorreria com '+
                   str(N_Maquinas_Reservas) +
                   ' máquinas reservas no valor de $' +
                   "{:.2f}".format(round(C_Min,2)).replace('.',',') +
                   '.')
        display(widgets.HTML(value = '<br><b>Resposta:</b><br>' + text))
        
button.on_click(on_button_clicked)

button_r.on_click(on_button_clicked_r)



In [8]:
header = widgets.HTML('<header><b> 03 - Atividade de máquina reserva  </b></header>')
display(header)
grid = GridspecLayout(2, 16,align_items='top', width='auto')
grid[0, :8] = matricula_l
grid[0, 8:16] = matricula
grid[1, :8] = button
grid[1, 8:16] = button_r
# grid[1,:15] = output
grid
#12.112.451-5

HTML(value='<header><h2> 03 - Atividade de máquina reserva  </h2></header>')

GridspecLayout(children=(HTML(value='Matricula:', layout=Layout(grid_area='widget001')), Text(value='12.112.45…

In [4]:
output

Output(layout=Layout(border='1px solid black'))

In [5]:
output.layout.visibility = 'hidden'